In [1]:
import nltk
import pandas as pd
import numpy as np
import pickle

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

### Load Data
#### Load "derecha diario" dataset from csv.

In [2]:
df = pd.read_csv('./data/derecha_econ_news.csv', index_col=0)

In [3]:
df.shape

(339, 6)

In [4]:
df.head(5)

,date,flyer,title,lead,body,url
0,30/11/2020,Economía,Economía española: en 12 meses se perdieron 10...,"Debido a la segunda ola del coronavirus, se pr...",Según el registro de la Seguridad Social españ...,https://derechadiario.com.ar/economia/economia...
1,29/11/2020,Economía,La enfermedad crónica que tiene Argentina: cas...,Desde la década de 1940 Argentina padece una e...,Argentina sirve como uno de los países más par...,https://derechadiario.com.ar/economia/la-enfer...
2,29/11/2020,Economía,El reconocido ejecutivo Mark Mobius recomienda...,El famoso hombre de las finanzas internacional...,"Mark Mobius, el reconocido gestor de mercados ...",https://derechadiario.com.ar/economia/el-recon...
3,28/11/2020,Economía,Ajuste a los jubilados: el aumento por decreto...,El Gobierno decretó un pobre aumento en torno ...,Alberto Fernández consolidó su reforma del sis...,https://derechadiario.com.ar/economia/ajuste-a...
4,27/11/2020,Economía,Brasil creó 395.000 puestos de trabajo en octu...,El Ministerio de Economía de Brasil confirmó u...,Con el acelerado ritmo de actividad en la econ...,https://derechadiario.com.ar/economia/brasil-c...


In [5]:
# remove unicode
df.body.replace('\n', ' ', regex=True ,inplace=True)
df.body.replace('\xa0', ' ', regex=True ,inplace=True)

### Topics
Selection of topics from news titles of the portal's economic section.

In [6]:
inflation = df.title[df.title.str.contains('inflacion|inflación|emisión|salarios|salario|indec')]
inflation.shape

(38,)

In [7]:
exchange = df.title[df.title.str.contains('dolar|dólar|dolares|dólares|dolarización|cambio|cepo')]
exchange.shape

(32,)

### Corpus (Body)
Clipping of the selected news body

#### Topic: Inflation

In [8]:
inflation = df.loc[df.title.str.contains('inflacion|inflación|emisión|salarios|salario|indec', na=False)]
inflation.shape

(38, 6)

In [9]:
inflation.date

1      29/11/2020
8      24/11/2020
20     12/11/2020
22     11/11/2020
24     09/11/2020
31     02/11/2020
41     24/10/2020
43     23/10/2020
54     15/10/2020
56     14/10/2020
58     12/10/2020
81     16/09/2020
88     11/09/2020
90     09/09/2020
97     01/09/2020
102    27/08/2020
113    14/08/2020
114    14/08/2020
132    21/07/2020
135    15/07/2020
165    12/06/2020
184    27/05/2020
190    22/05/2020
196    17/05/2020
198    15/05/2020
206    11/05/2020
210    08/05/2020
215    06/05/2020
221    01/05/2020
223    30/04/2020
226    28/04/2020
233    22/04/2020
241    18/04/2020
247    15/04/2020
266    06/04/2020
272    02/04/2020
298    17/03/2020
305    12/03/2020
Name: date, dtype: object

In [10]:
inflation.dropna(inplace=True)
inflation.shape

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(36, 6)

In [11]:
inflation = inflation.reset_index()

In [12]:
inflation.insert(loc=1, column='topics', value='inflation')

In [13]:
inflation.head(5)

,index,topics,date,flyer,title,lead,body,url
0,1,inflation,29/11/2020,Economía,La enfermedad crónica que tiene Argentina: cas...,Desde la década de 1940 Argentina padece una e...,Argentina sirve como uno de los países más par...,https://derechadiario.com.ar/economia/la-enfer...
1,8,inflation,24/11/2020,Economía,El economista Steve Hanke advierte que la infl...,El economista norteamericano de la Universidad...,"El economista Steve Hanke, profesor e investig...",https://derechadiario.com.ar/economia/el-econo...
2,20,inflation,12/11/2020,Economía,Se acelera el aumento de precios: la inflación...,El Índice de Precios al Consumidor aumentó un ...,El INDEC divulgó las cifras oficiales para el ...,https://derechadiario.com.ar/economia/se-acele...
3,22,inflation,11/11/2020,Economía,Socialismo y miseria: la economía venezolana c...,El régimen del dictador Maduro continúa destru...,Desde la toma ilegal del poder de Maduro hace ...,https://derechadiario.com.ar/economia/socialis...
4,24,inflation,09/11/2020,Economía,“Efecto Lipovetzky”: los alquileres siguen aum...,"La iniciativa del PRO, formulada originalmente...",En un año marcado por las nuevas regulaciones ...,https://derechadiario.com.ar/economia/efecto-l...


#### Topic: Currency Exchange

In [14]:
exchange = df.loc[df.title.str.contains('dolar|dólar|dolares|dólares|dolarización|cambio|cepo', na=False)]
exchange.shape

(32, 6)

In [15]:
exchange.date

2      29/11/2020
8      24/11/2020
13     18/11/2020
18     13/11/2020
39     26/10/2020
42     23/10/2020
49     19/10/2020
55     15/10/2020
60     09/10/2020
63     07/10/2020
65     05/10/2020
68     02/10/2020
74     25/09/2020
82     16/09/2020
83     15/09/2020
90     09/09/2020
113    14/08/2020
128    25/07/2020
145    03/07/2020
166    11/06/2020
171    07/06/2020
181    29/05/2020
218    04/05/2020
227    27/04/2020
228    27/04/2020
232    23/04/2020
242    17/04/2020
243    17/04/2020
248    14/04/2020
267    04/04/2020
307    12/03/2020
331    23/02/2020
Name: date, dtype: object

In [16]:
exchange.dropna(inplace=True)
exchange.shape

/home/guido/anaconda3/envs/dh/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(31, 6)

In [17]:
exchange = exchange.reset_index()

In [18]:
exchange.insert(loc=1, column='topics', value='exchange')

In [19]:
exchange.head(5)

,index,topics,date,flyer,title,lead,body,url
0,2,exchange,29/11/2020,Economía,El reconocido ejecutivo Mark Mobius recomienda...,El famoso hombre de las finanzas internacional...,"Mark Mobius, el reconocido gestor de mercados ...",https://derechadiario.com.ar/economia/el-recon...
1,8,exchange,24/11/2020,Economía,El economista Steve Hanke advierte que la infl...,El economista norteamericano de la Universidad...,"El economista Steve Hanke, profesor e investig...",https://derechadiario.com.ar/economia/el-econo...
2,13,exchange,18/11/2020,Economía,Desesperación en el régimen chavista: el Banco...,"Tras casi cinco años de hiperinflación, el Ban...","Según un reciente informe de Bloomberg, fuente...",https://derechadiario.com.ar/economia/desesper...
3,18,exchange,13/11/2020,Economía,Nueva subida del dólar blue: ya acumula $23 en...,"Tras haber caído por debajo de los $150, el dó...",Hoy el dólar libre llegó a venderse a $174 en ...,https://derechadiario.com.ar/economia/nueva-su...
4,39,exchange,26/10/2020,Economía,El Gobierno lanza una mega-licitación de pesos...,El ministro Guzmán preparó una serie de instru...,Después de casi dos semanas de constantes aume...,https://derechadiario.com.ar/economia/el-gobie...


In [20]:
data_clean = inflation.append(exchange)
data_clean.shape

(67, 8)

I drop those news labeled under both topics

In [21]:
data_clean = data_clean.drop_duplicates(subset='index',keep=False)
data_clean.shape

(61, 8)

In [22]:
print('num inflation news:', data_clean.loc[data_clean.topics == 'inflation',].shape[0])
print('num exchange news:', data_clean.loc[data_clean.topics == 'exchange',].shape[0])

num inflation news: 33
num exchange news: 28


Check the number of words per article's body and discard those incompleted

In [23]:
data_clean.body.apply(lambda x: len(word_tokenize(x))) < 300

0     False
2     False
3     False
4     False
5     False
      ...  
26     True
27    False
28     True
29     True
30    False
Name: body, Length: 61, dtype: bool

In [24]:
filter = data_clean.body.apply(lambda x: len(word_tokenize(x))) < 300
data_clean.drop(data_clean[filter].index, axis=0, inplace=True)

#### Save selection

In [25]:
data_clean.reset_index(drop=True, inplace=True)

In [26]:
data_clean.drop('index', axis=1, inplace=True)

In [27]:
data_clean.sample(5)

,topics,date,flyer,title,lead,body,url
16,inflation,15/07/2020,Economía,"Con la economía postrada, la inflación de juni...","Según el informe presentado hoy por el INDEC, ...",El Índice de Precios al Consumidor (IPC) que m...,https://derechadiario.com.ar/economia/con-la-e...
9,inflation,12/10/2020,Economía,Un dato que asusta: la inflación de septiembre...,El consenso entre las consultoras privadas hab...,"Esta semana, el INDEC anunciará el dato oficia...",https://derechadiario.com.ar/economia/un-dato-...
0,inflation,29/11/2020,Economía,La enfermedad crónica que tiene Argentina: cas...,Desde la década de 1940 Argentina padece una e...,Argentina sirve como uno de los países más par...,https://derechadiario.com.ar/economia/la-enfer...
22,inflation,01/05/2020,Economía,Domingo Cavallo alerta que la devaluación será...,El famoso ex-ministro Cavallo aseguró que la d...,Domingo Cavallo proporcionó una entrevista org...,https://derechadiario.com.ar/economia/domingo-...
32,exchange,26/10/2020,Economía,El Gobierno lanza una mega-licitación de pesos...,El ministro Guzmán preparó una serie de instru...,Después de casi dos semanas de constantes aume...,https://derechadiario.com.ar/economia/el-gobie...


In [28]:
pickle.dump(data_clean, open( "data/df_clean_der.pkl", "wb" ))

In [29]:
corpus = data_clean
corpus.shape

(53, 7)

### Tokenization
#### Manual

In [30]:
import re
import unidecode

"Stop words", special characters, accents and numbers are removed

In [31]:
stopwords_sp = stopwords.words('spanish')

In [32]:
def tokenizer(texto):

    alphanumeric = re.sub(r'([^\s\w]|_)+', '', texto).lower()
    no_accents = unidecode.unidecode(alphanumeric)
    
    tockens = word_tokenize(no_accents)
    
    tockens_clean = [tocken for tocken in tockens if tocken not in stopwords_sp and tocken.isalpha()]
    
    terminos = tockens_clean

    return terminos

In [33]:
tokenized = [tokenizer(documento) for documento in corpus.body]

In [34]:
vocabulario = sorted(list(set([word for group in tokenized for word in group])))

In [35]:
len(vocabulario)

4555

#### CountVectorizer

In [36]:
from sklearn.feature_extraction.text import CountVectorizer

Removal of special characters function to load in vectorizer

In [37]:
def removal(text):
    text = re.sub(r'(\d|\$|\%|\+)', '', text.lower())
    return re.sub(r'\d+', '', text)

In [38]:
vectorizer = CountVectorizer(stop_words = stopwords_sp, lowercase = True, strip_accents='unicode', preprocessor=removal)

In [39]:
vectorizer.fit(corpus.body)

CountVectorizer(preprocessor=<function removal at 0x7f6c89b4f830>,
                stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...],
                strip_accents='unicode')

In [40]:
len(vectorizer.vocabulary_), len(vectorizer.get_feature_names())

(4414, 4414)

I prefare the manual tokenization results since the process is clearer and there are more terms.

### Pickle

Manual tokens

In [41]:
def token_matrix_maker(data, vocabulario):
    matriz = np.zeros(shape = (len(data), len(vocabulario)), dtype='int')
    for i, documento in enumerate(data):
        for termino in documento:
            matriz[i, vocabulario.index(termino)] += 1
    return matriz

In [42]:
matrix = token_matrix_maker(tokenized, vocabulario)

In [43]:
# make dataframe
token_df = pd.DataFrame(matrix, columns=vocabulario, index = ['doc' + str(i + 1) for i in range(len(tokenized))] )

In [44]:
# add classification column
topics = ['inflation' for i in range((corpus.topics == 'inflation').sum())] + ['exchange' for i in range((corpus.topics == 'exchange').sum())]
token_df.insert(loc=0, column='topics', value=topics)

In [45]:
token_df

,topics,abajo,abandonara,abandone,abastecimiento,abatida,abierta,abierto,aboga,abraza,...,woyecheszen,xx,yaguarete,york,ypf,yuanes,zamora,zona,zonaprop,zonas
doc1,inflation,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
doc2,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
doc3,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc4,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,2,0
doc5,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
doc6,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc7,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc8,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc9,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc10,inflation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
token_df.to_pickle('tokens/topic_tokens_der.pkl')

#### CV tokens

In [47]:
pickle.dump(vectorizer, open( "tokens/cv_der.pkl", "wb" ))